<a href="https://colab.research.google.com/github/2003Yash/langmem_implementation/blob/main/langmem_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source: https://www.youtube.com/watch?v=3Yp-hIEcWXk&list=PLZAGXXsIV3P3gCenOWRd56ZpeksdYFUKV&index=3

Step-1: Import dependencies

In [ ]:
!pip install -U langmem langgraph

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

Step-2: Normal Agentic Memory ( Short-Term Memory => Session Dependent )

In [ ]:
# Initialize Memory and Agent
checkpointer = InMemorySaver()
store = InMemoryStore()

agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=[], store=store, checkpointer=checkpointer)

In [ ]:
# Generic Chat LLM Function
def chat(agent, txt, thread_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]}, config={"configurable": {"thread_id": thread_id}})
    return result_state["messages"][-1].content

In [ ]:
thread_1 = "thread-1"

chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)

chat(agent, "Nice! Wish me luck!", thread_1)


# Memory is LOST in new thread since Memory in Session Dependent

thread_2 = "thread-2"

chat(agent, "Nice! Oh thank you! It'll be hard.", thread_2)

Step-3: Adding Long Term Memory ( Session Independent = Langmem Stores important parts of conversation into embedding of 136 dims as given below an stores in vecotr data and when user asks a query it returns most similar results. Essentialy a RAG )

In [ ]:
# Just import Memory from Langmem and Use it as tools for our Chatbot

from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()
agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=memory_tools, store=store, checkpointer=checkpointer)

In [ ]:
# Here it remembers context even after thread_id is changed

thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)
print(chat(agent, "Nice! Wish me luck! Please note down the detailed memories for me :)", thread_1))


thread_2 = "thread-2"
chat(agent, "Remember what I'm supposed to do for my training this week? It's week 3...", thread_2)
chat(agent, "That may be tricky. I just sprained my ankle. Could you update my plan to include more cross training? Be sure to update the existing key of our plan", thread_2)

Step-4: Creating User wise Long Term Memory ( just add user_id in namespace when creating memory tools from langmem)

In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories", "{user_id}")
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=memory_tools, store=store, checkpointer=checkpointer)

In [ ]:
def chat(agent, txt, thread_id, user_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]},
                                config={"configurable": {"thread_id": thread_id, "user_id": user_id}})
    return result_state["messages"][-1].content

In [ ]:

thread_1 = "thread-1"
user_id = "User-A"
chat(agent,
     "Hi I'm Will, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare and help me stay honest??",
     thread_1,
     user_id)

# As soon as User_id is changed memory is seperated

thread_1 = "thread-2"
user_id2 = "User-B"
chat(agent,
     "Hi I'm John, I'm learning chess - could you help me become great??",
     thread_1,
     user_id2)

In [ ]:
# print User-wise Memories

items = store.search(("agent_memories",))
for item in items:
    print(item.namespace, item.value)

Step-5: "Eager" memory retrieval ( We can fetch memories before the first LLM call to simplify its response. Otherwise, it has known and unknown, unknowns so will almost always try to search for some subclass of questions )

In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph.config import get_store

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

def prompt(state):
    # Search over memories based on the messages
    store = get_store()
    items = store.search(namespace, query=state["messages"][-1].content)
    memories = "\n\n".join(str(item) for item in items)
    system_msg = {"role": "system", "content": f"## Memories:\n\n{memories}"}
    return [system_msg] + state["messages"]

agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", prompt=prompt, tools=memory_tools, store=store, checkpointer=checkpointer)

In [ ]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1, None)

In [ ]:
print(chat(agent, "Nice! Wish me luck! Please note down the detailed memories for me :)", thread_1, None))

In [ ]:
thread_2 = "thread-2"
chat(agent, "What I'm supposed to do for my training this week? It's week 3...", thread_2, None)